In [ ]:
#installing ollama on host
!curl https://ollama.ai/install.sh | sh

In [ ]:
# install dependencies for python script
!pip install pyngrok

import os
import asyncio
from google.colab import userdata

# set LD_LIBRARY_PATH so system NVIDIA library becomes preferred
# over the built-in library. This is particularly important for
# Google Colab which installs older drivers
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd):
    '''
    run is a helper function to run subcommands asynchronously.
    '''
    # create subprocess and collect the output
    print('>>> starting', *cmd)
    p = await asyncio.subprocess.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE,
    )

    # print lines utf-8 encoded
    async def pipe(lines):
      async for line in lines:
        print(line.strip().decode('utf-8'))

    await asyncio.gather(
        pipe(p.stdout),
        pipe(p.stderr),
    )

In [ ]:
token = userdata.get("NGROKTOKEN")
from pyngrok import ngrok
!ngrok config add-authtoken {token}
# !cat /root/.config/ngrok/ngrok.yml

In [ ]:
# install streamlit for app ui along with additional required packages
!pip install -q streamlit langchain langchain_community BeautifulSoup4 tiktoken chromadb

In [ ]:
# run ollama sub process, then run streamlit app and expose it via ngrok
# streamlit app should be upploaded to files under /content
await asyncio.gather(
    run_process(['ollama', 'serve']),
    run_process(['ollama', 'pull', 'nomic-embed-text']),
    run_process(['ollama', 'run', 'mistral']),
    run_process(['streamlit', 'run', 'app.py']),
    run_process(['ngrok', 'http', '--log', 'stderr', '8501']),
)


In [ ]:
# run ollama sub process and expose it from localhost:11434 to public via http
# await asyncio.gather(
#     run_process(['ollama', 'serve']),
#     run_process(['ngrok', 'http', '--log', 'stderr', '11434', '--host-header="localhost:11434", '--request-header="localhost:11434"']),
# )

# !ollama serve



In [ ]:
# This line will shut down the ngrok agent
ngrok.kill()

# https://github.com/ollama/ollama/blob/main/examples/jupyter-notebook/ollama.ipynb

# https://stackoverflow.com/questions/77697302/how-to-run-ollama-in-google-colab